In [22]:
import numpy as np
import pandas as pd

from geopy.distance import vincenty
import datetime
import time

### Введение

#### Загрузите csv файл в датафрейм

In [11]:
citybike = pd.read_csv('201809-citibike-tripdata.csv')

In [8]:
citybike.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,284,2018-09-01 01:48:22.0320,2018-09-01 01:53:06.9130,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,26303,Subscriber,1984,1
1,370,2018-09-01 01:48:22.5010,2018-09-01 01:54:32.9270,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,29667,Subscriber,1984,1
2,1181,2018-09-01 09:37:04.3070,2018-09-01 09:56:45.3280,3183,Exchange Place,40.716247,-74.033459,3192,Liberty Light Rail,40.711242,-74.055701,26307,Customer,1994,1
3,383,2018-09-01 09:44:46.0030,2018-09-01 09:51:09.9230,3183,Exchange Place,40.716247,-74.033459,3213,Van Vorst Park,40.718489,-74.047727,29261,Subscriber,1987,1
4,377,2018-09-01 09:44:46.1850,2018-09-01 09:51:03.6330,3183,Exchange Place,40.716247,-74.033459,3213,Van Vorst Park,40.718489,-74.047727,33665,Subscriber,1985,1


1. Найти общее количество строк и столбцов в датасете - указать первым число строк, вторым - число столбцов

In [12]:
citybike.shape

(1877884, 15)

Найти среднюю длину поездок в минутах(столбец tripduration) c точностью до 2 знака

In [13]:
citybike['tripduration'] = citybike['tripduration']/60
round(citybike.describe()['tripduration']['mean'], 2)

16.13

Сколько поездок начались и закончились в той же самой станции?

In [14]:
citybike[(citybike['start station id'] == citybike['end station id'])].shape

(41364, 15)

Сколько поездок начались и закончились в той же самой станции? Указать количество уникальных байков

In [43]:
citybike[(citybike['start station id'] == citybike['end station id'])]['bikeid'].unique().shape

(9939,)

Какой самый используемый байк(bikeid) в городе по количеству поездок? 

In [19]:
citybike['bikeid'].value_counts()[:1]

33875    559
Name: bikeid, dtype: int64

Найдите байк(bikeid), у которого в среднем продолжительность поездок выше, чем у всех остальных

In [15]:
citybike.groupby('bikeid')['tripduration'].mean().sort_values(ascending=False)[:1]

bikeid
17548    8133.706667
Name: tripduration, dtype: float64

Найдите количество строк, в которых отсутствуют данные о start station id

In [17]:
citybike[citybike['start station id'].isnull()].shape

(716, 15)

Какова средняя продолжительность поездки в зависимости от типа подписки c точностью до 2 знака? 

In [18]:
round(citybike.groupby('usertype')['tripduration'].mean(),2)

usertype
Customer      33.42
Subscriber    13.33
Name: tripduration, dtype: float64

Для каждой станции найдите расстояние между станциями, а затем найдите среднее расстояние по всем поездкам, предварительно выкинув замкнутые траектории(те у которых совпадают start station id = end station id). 

Hint: можно воспользоваться библиотекой geopy и взять расстояние vincenty(минимальное расстояние между точками)


In [12]:
citybike['distance_km'] = citybike.apply(lambda x: vincenty((x['start station latitude'], x['start station longitude']),
                                                            (x['end station latitude'], x['end station longitude'])).kilometers, axis=1)

print(citybike['distance_km'].mean())



In [15]:
citybike['end station name'].value_counts()[:5]

Grove St PATH     2563
Exchange Place    1128
Hamilton Park     1074
Sip Ave            814
Newport PATH       721
Name: end station name, dtype: int64

Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера

In [24]:
citybike['end_hour'] = citybike['stoptime'].apply(lambda x: datetime.datetime.fromtimestamp(
            time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [25]:
citybike['start_hour'] = citybike['starttime'].apply(lambda x: datetime.datetime.fromtimestamp(
                                        time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [33]:
citybike[(citybike.start_hour.isin([18, 19, 20]))]['start station id'].value_counts().head()

519.0    3181
426.0    2701
402.0    2680
459.0    2605
477.0    2418
Name: start station id, dtype: int64

Выберите станции(end station id), в которые приезжают с 6 до 10 утра


In [29]:
end_stations = [3140, 3106, 3116, 369]

In [30]:
citybike[(citybike.end_hour.isin([6,7,8,9,10])) &
         (citybike['end station id'].isin(end_stations))]['end station id'].unique()

array([3106., 3116., 3140.,  369.])